# Triplet Loss

참고) https://github.com/andreasveit/triplet-network-pytorch

### 설계
- 두 방향의 triplet loss를 고려하고 있다.
    1. 같은 폰트를 구분하는가
    2. 같은 글자를 구분하는가 <br/>
    
    
2번의 경우 데이터가 많이 부족하다. (같은 글자는 전체 데이터셋에서 107개 뿐) <br/>
그러나 MNIST와 유사한 형태이기 때문에 데이터 수만 확보된다면 가능할 것 같긴 하다.


1번의 경우 각 폰트마다 2350개의 글자가 있기 때문에 이 방향으로 구현할 것이다.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Tripletnet(nn.Module):
    def __init__(self, embeddingnet):
        super(Tripletnet, self).__init__()
        self.embeddingnet = embeddingnet

    def forward(self, x, y, z):
        embedded_x = self.embeddingnet(x)
        embedded_y = self.embeddingnet(y)
        embedded_z = self.embeddingnet(z)
        dist_a = F.pairwise_distance(embedded_x, embedded_y, 2)
        dist_b = F.pairwise_distance(embedded_x, embedded_z, 2)
        return dist_a, dist_b, embedded_x, embedded_y, embedded_z

## Triplet list 만들기
Triplet loss를 사용하기 위해서는 데이터셋을 triplet의 형태에 맞게 바꾸어야 한다. <br/>
https://github.com/andreasveit/triplet-network-pytorch/blob/master/triplet_mnist_loader.py


- triplet_list.txt 파일을 만들어야 한다.
- 한 줄당 3개의 index로 구성되어 있다.
- 각 index는 [anchor, negative, positive] 이미지의 인덱스이다.

In [5]:
from common.dataset import PickledImageProvider
from common.dataset import KoreanFontDataset

In [6]:
dset_train = KoreanFontDataset(PickledImageProvider('dataset/kor/train.obj'), vector_size=10)

processed 1000 examples
processed 2000 examples
processed 3000 examples
processed 4000 examples
processed 5000 examples
processed 6000 examples
processed 7000 examples
processed 8000 examples
processed 9000 examples
processed 10000 examples
processed 11000 examples
processed 12000 examples
processed 13000 examples
processed 14000 examples
processed 15000 examples
processed 16000 examples
processed 17000 examples
processed 18000 examples
processed 19000 examples
processed 20000 examples
processed 21000 examples
processed 22000 examples
processed 23000 examples
processed 24000 examples
processed 25000 examples
processed 26000 examples
processed 27000 examples
processed 28000 examples
processed 29000 examples
processed 30000 examples
processed 31000 examples
processed 32000 examples
processed 33000 examples
processed 34000 examples
processed 35000 examples
processed 36000 examples
processed 37000 examples
processed 38000 examples
processed 39000 examples
processed 40000 examples
processed

In [9]:
dset_train[0][0]['font_index']

0

모든 데이터셋에서 font_index만 추출한다.

In [12]:
font_indexes = []
for i in range(len(dset_train)):
    font_indexes.append(dset_train[i][0]['font_index'])

In [18]:
font_indexes = np.array(font_indexes)

In [19]:
len(font_indexes)

202100

In [36]:
def make_triplet_list(ntriplets): # 여기서 ntriplets는 202100 (train size)
    print('Processing Triplet Generation ...')
    np_labels = font_indexes # todo: 맞게 변형하기 (모든 데이터셋 중 특정 폰트 인덱스를 추출할 수 있을까?)
    filename = 'triplet_list.txt'
 
    triplets = []
    for class_idx in range(86): # train font size        
        try:
            a = np.random.choice(np.where(np_labels==class_idx)[0], int(ntriplets/86), replace=True) # 2350 anchor
            b = np.random.choice(np.where(np_labels==class_idx)[0], int(ntriplets/86), replace=True) # 2350 positive
            while np.any((a-b)==0):
                np.random.shuffle(b)
            c = np.random.choice(np.where(np_labels!=class_idx)[0], int(ntriplets/10), replace=True) # 2350 negative

            for i in range(a.shape[0]):
                triplets.append([int(a[i]), int(c[i]), int(b[i])]) # [anchor, negative, positive]           
        except:
            print('exception font index:', class_idx)
    with open(os.path.join('./', filename), "w") as f:
        writer = csv.writer(f, delimiter=' ')
        writer.writerows(triplets)
    print('Done!')

In [37]:
make_triplet_list(202100)

Processing Triplet Generation ...
exception font index: 8
exception font index: 9
exception font index: 81
exception font index: 82
exception font index: 83
exception font index: 84
exception font index: 85
Done!


## TrainLoader 커스텀
MNIST DataLoader를 커스텀하여 사용하기로 한다. <br/>
https://github.com/andreasveit/triplet-network-pytorch/blob/master/triplet_image_loader.py

In [3]:
from PIL import Image
import os
import os.path

import torch.utils.data
import torchvision.transforms as transforms

def default_image_loader(path):
    return Image.open(path).convert('RGB')

class TripletImageLoader(torch.utils.data.Dataset): # todo: init에 Pickled 클래스 포함시키기
    def __init__(self, base_path, filenames_filename, triplets_file_name, transform=None,
                 loader=default_image_loader):
        """ 
        filenames_filename: 
            A text file with each line containing the path to an image e.g.,
            images/class1/sample.jpg
                
        triplets_file_name: 
            A text file with each line containing three integers, 
            where integer i refers to the i-th image in the filenames file. 
            For a line of intergers 'a b c', a triplet is defined such that image a is more 
            similar to image c than it is to image b, 
            e.g., 0 2017 42 
        """
        self.base_path = base_path  
        self.filenamelist = []
        for line in open(filenames_filename):
            self.filenamelist.append(line.rstrip('\n'))
        triplets = []
        for line in open(triplets_file_name):
            triplets.append((line.split()[0], line.split()[1], line.split()[2])) # anchor, far, close
        self.triplets = triplets
        self.transform = transform
        self.loader = loader

    def __getitem__(self, index):
        path1, path2, path3 = self.triplets[index]
        img1 = self.loader(os.path.join(self.base_path,self.filenamelist[int(path1)]))
        img2 = self.loader(os.path.join(self.base_path,self.filenamelist[int(path2)]))
        img3 = self.loader(os.path.join(self.base_path,self.filenamelist[int(path3)]))
        if self.transform is not None:
            img1 = self.transform(img1)
            img2 = self.transform(img2)
            img3 = self.transform(img3)

        return img1, img2, img3

    def __len__(self):
        return len(self.triplets)